<a href="https://colab.research.google.com/github/Kyllien/DRIM-Game/blob/main/MatriceTransitionGood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#Import data from os
import os
os.chdir("/content/gdrive/My Drive/DRIM")

In [4]:
#Librairie
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [5]:
data = pd.read_csv('base_1.csv')

# print(np.unique(data['classe'])) #Aucun prob de classe

#rename the columns
data = data.rename(columns={"classe": "State", "dtf_per_trt": "Time", "ID_BCR_TRS": "ID"})
# Put Time in Date Forma
data["Time"]=pd.to_datetime(data["Time"])



In [ ]:
#Creation du Jeu Test
df = data.sample(10)
#df.to_csv('C:/Users/33640/source/repos/Kyllien/DRIM-Game/data_sample.csv')

#df = data.head(1000000)
df = df.sort_values(by=["ID","Time"])

In [22]:
df=data
df=df.sort_values(by=["Time"])
ID=np.unique(df["ID"])
df2=pd.DataFrame(ID,columns=["ID"])
compteur=0
for i in np.unique(df["Time"]):
  print(i)
  df3=df[df["Time"]==i]
  df3=df3[["ID","State"]]
  df3=df3.rename({'ID': 'ID', 'State': 'Date'+str(compteur)}, axis='columns')
  compteur=compteur+1
  df2=df2.merge(df3,on='ID',how='left')


2010-01-31T00:00:00.000000000
2010-02-28T00:00:00.000000000
2010-03-31T00:00:00.000000000
2010-04-30T00:00:00.000000000
2010-05-31T00:00:00.000000000
2010-06-30T00:00:00.000000000
2010-07-31T00:00:00.000000000
2010-08-31T00:00:00.000000000
2010-09-30T00:00:00.000000000
2010-10-31T00:00:00.000000000
2010-11-30T00:00:00.000000000
2010-12-31T00:00:00.000000000
2011-01-31T00:00:00.000000000
2011-02-28T00:00:00.000000000
2011-03-31T00:00:00.000000000
2011-04-30T00:00:00.000000000
2011-05-31T00:00:00.000000000
2011-06-30T00:00:00.000000000
2011-07-31T00:00:00.000000000
2011-08-31T00:00:00.000000000
2011-09-30T00:00:00.000000000
2011-10-31T00:00:00.000000000
2011-11-30T00:00:00.000000000
2011-12-31T00:00:00.000000000
2012-01-31T00:00:00.000000000
2012-02-29T00:00:00.000000000
2012-03-31T00:00:00.000000000
2012-04-30T00:00:00.000000000
2012-05-31T00:00:00.000000000
2012-06-30T00:00:00.000000000
2012-07-31T00:00:00.000000000
2012-08-31T00:00:00.000000000
2012-09-30T00:00:00.000000000
2012-10-31

In [104]:
#Creation Matrice PIT sans les NR(12)
PIT={}
for i in range(1,96):
  u=df2.columns[i]
  m=df2.columns[i+1]
  PIT[i]=pd.crosstab(df2[u],df2[m],normalize="index",dropna=True)
  for j in range(1,11):
    PIT[i].loc[11,j] = 0
  PIT[i].loc[11,11] = 1

In [27]:
#Creation matrice TTC sans les NR(12)
TTC=PIT[1]
for i in range(2,96):
  TTC=TTC+PIT[i]
TTC=TTC/len(PIT)
TTC

Date1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
Date0,,,,,,,,,,,
1.0,0.970357,0.014852,0.011744,0.002345,0.000176,0.000383,0.000069,0.000043,0.000023,0.000003,0.000007
2.0,0.025838,0.959496,0.001567,0.002870,0.004158,0.004869,0.001023,0.000079,0.000085,0.000004,0.000010
3.0,0.004818,0.027760,0.955731,0.000341,0.000384,0.004316,0.004964,0.001376,0.000190,0.000101,0.000019
4.0,0.000060,0.013048,0.030364,0.940812,0.000481,0.000699,0.011099,0.000105,0.003089,0.000171,0.000073
5.0,0.000059,0.009819,0.008680,0.022321,0.940846,0.000664,0.012350,0.000420,0.004502,0.000228,0.000113
6.0,0.000074,0.000932,0.014678,0.001824,0.015822,0.940915,0.001811,0.014090,0.006834,0.002751,0.000269
7.0,0.000040,0.000950,0.002507,0.008902,0.009430,0.017109,0.917311,0.001896,0.022824,0.017997,0.001034
8.0,0.000040,0.000387,0.006037,0.001126,0.001179,0.043849,0.036728,0.832132,0.014491,0.060491,0.003540
9.0,0.000041,0.000682,0.001313,0.006178,0.009493,0.024608,0.090554,0.020750,0.707087,0.121018,0.018276


In [30]:
#TTC Cumule sans les NR(12)
TTC_C = TTC
TTC[11][1]
for i in range(1,12):
  compteur=0
  for j in reversed(range(1,12)):
    compteur=TTC[j][i]+compteur
    TTC_C[j][i]=compteur
TTC_C

Date1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
Date0,,,,,,,,,,,
1.0,1.0,0.029643,0.014791,0.003048,0.000703,0.000527,0.000145,0.000076,0.000033,0.000009,0.000007
2.0,1.0,0.974162,0.014665,0.013098,0.010228,0.006070,0.001202,0.000179,0.000100,0.000014,0.000010
3.0,1.0,0.995182,0.967422,0.011691,0.011350,0.010966,0.006650,0.001686,0.000310,0.000120,0.000019
4.0,1.0,0.999940,0.986892,0.956528,0.015716,0.015236,0.014537,0.003438,0.003333,0.000244,0.000073
5.0,1.0,0.999941,0.990122,0.981442,0.959121,0.018275,0.017611,0.005262,0.004842,0.000340,0.000113
6.0,1.0,0.999926,0.998994,0.984316,0.982491,0.966669,0.025755,0.023943,0.009854,0.003019,0.000269
7.0,1.0,0.999960,0.999010,0.996503,0.987601,0.978171,0.961063,0.043751,0.041855,0.019031,0.001034
8.0,1.0,0.999960,0.999573,0.993536,0.992409,0.991230,0.947381,0.910653,0.078522,0.064030,0.003540
9.0,1.0,0.999959,0.999277,0.997964,0.991786,0.982293,0.957685,0.867131,0.846381,0.139294,0.018276


In [31]:
#Creation d'une classe pour les non rating : 12
df3=df2
df3=df3.fillna(12)

In [32]:
#Creation Matrice PIT avec les NR(12)
PIT_NR={}
for i in range(1,96):
  u=df3.columns[i]
  m=df3.columns[i+1]
  PIT_NR[i]=pd.crosstab(df3[u],df3[m],normalize="index")
  PIT_NR[i][11:][range(1,10)] = 0
  PIT_NR[i][11:][11] = 1
  for j in range(1,11):
    PIT_NR[i].loc[11,j] = 0
  PIT_NR[i].loc[11,11] = 1



In [33]:
#Creation matrice TTC avec les NR(12)
TTC_NR=PIT_NR[1]
for i in range(2,96):
  TTC_NR=TTC_NR+PIT_NR[i]
TTC_NR=TTC_NR/len(PIT_NR)
TTC_NR

Date1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
Date0,,,,,,,,,,,,
1.0,0.918037,0.014044,0.011101,0.002218,0.000166,0.000362,0.000065,0.000041,0.000022,0.000002,0.000007,0.053935
2.0,0.025523,0.947654,0.001547,0.002833,0.004105,0.004805,0.001010,0.000078,0.000085,0.000004,0.000010,0.012347
3.0,0.004771,0.027495,0.946670,0.000338,0.000381,0.004274,0.004916,0.001363,0.000188,0.000100,0.000019,0.009486
4.0,0.000059,0.012926,0.030076,0.931962,0.000476,0.000693,0.010990,0.000104,0.003059,0.000169,0.000072,0.009413
5.0,0.000059,0.009737,0.008609,0.022137,0.933075,0.000658,0.012246,0.000416,0.004465,0.000226,0.000112,0.008261
6.0,0.000073,0.000923,0.014535,0.001806,0.015672,0.931821,0.001794,0.013944,0.006767,0.002724,0.000266,0.009676
7.0,0.000040,0.000939,0.002479,0.008797,0.009326,0.016917,0.906917,0.001875,0.022558,0.017797,0.001022,0.011334
8.0,0.000039,0.000383,0.005972,0.001114,0.001166,0.043384,0.036326,0.823006,0.014334,0.059834,0.003499,0.010943
9.0,0.000040,0.000673,0.001296,0.006093,0.009364,0.024286,0.089396,0.020483,0.697799,0.119445,0.018033,0.013092


In [ ]:
#Calcul des coefficient de correlation
dataDefaut = pd.read_excel("Taux.Defaut.xlsx")
dataDefaut


,Time,Taux_Defaut
0,2010-01-31,0.059661
1,2010-02-28,0.052415
2,2010-03-31,0.051614
3,2010-04-30,0.049995
4,2010-05-31,0.049792
...,...,...
115,2019-08-31,0.029779
116,2019-09-30,0.027923
117,2019-10-31,0.027948
118,2019-11-30,0.027663


In [34]:
#TTC Cumule avec les NR(12)
TTC_NR_C = TTC_NR
TTC_NR[12][1]
for i in range(1,13):
  compteur=0
  for j in reversed(range(1,13)):
    compteur=TTC_NR[j][i]+compteur
    TTC_NR_C[j][i]=compteur
TTC_NR_C


Date1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
Date0,,,,,,,,,,,,
1.0,1.0,0.081963,0.067918,0.056817,0.054600,0.054434,0.054072,0.054007,0.053966,0.053944,0.053942,0.053935
2.0,1.0,0.974477,0.026823,0.025277,0.022444,0.018339,0.013533,0.012523,0.012445,0.012360,0.012357,0.012347
3.0,1.0,0.995229,0.967734,0.021064,0.020727,0.020346,0.016072,0.011156,0.009793,0.009605,0.009505,0.009486
4.0,1.0,0.999941,0.987015,0.956938,0.024976,0.024500,0.023807,0.012817,0.012713,0.009654,0.009485,0.009413
5.0,1.0,0.999941,0.990204,0.981596,0.959458,0.026384,0.025726,0.013480,0.013064,0.008599,0.008373,0.008261
6.0,1.0,0.999927,0.999004,0.984470,0.982663,0.966992,0.035171,0.033377,0.019433,0.012666,0.009943,0.009676
7.0,1.0,0.999960,0.999022,0.996543,0.987746,0.978420,0.961503,0.054586,0.052711,0.030153,0.012356,0.011334
8.0,1.0,0.999961,0.999578,0.993606,0.992492,0.991326,0.947942,0.911616,0.088610,0.074277,0.014443,0.010943
9.0,1.0,0.999960,0.999286,0.997990,0.991897,0.982533,0.958247,0.868851,0.848369,0.150570,0.031125,0.013092
